## Feature Selection - Activities

In [1]:
# Data Management/Investigation
import pandas as pd
import numpy as np
import missingno as miss
from plotnine import *
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# For pre-processing data 
from sklearn import preprocessing as pp 
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest, chi2

# For splits and CV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold # Cross validation 
from sklearn.model_selection import cross_validate # Cross validation 
from sklearn.model_selection import GridSearchCV # Cross validation + param. tuning.

# Machine learning methods 
from sklearn.linear_model import LinearRegression as LM
from sklearn.neighbors import KNeighborsRegressor as KNN
from sklearn.tree import DecisionTreeRegressor as DTree
from sklearn import tree # For plotting the decision tree rules
from sklearn.ensemble import BaggingRegressor as Bag
from sklearn.ensemble import RandomForestRegressor as RF

# For evaluating our model's performance
import sklearn.metrics as m

# Pipeline to combine modeling elements
from sklearn.pipeline import Pipeline

In [2]:
dta = pd.read_csv('data/clean_data.csv')
dta.columns

Index(['Participant_ID', 'Category', 'Age_yrs', 'Age_category',
       'Education_yrs', 'Education_category', 'Sex', 'Marital_status',
       'Income', 'Occupation_class', 'Living_area', 'BP_sistol', 'BP_diastol',
       'BMI', 'GDS', 'Glucose', 'Triglyceride', 'HDL', 'Hypertension',
       'BMI_category', 'Diabetes', 'Tri_200', 'HDL_40', 'Smoking_status',
       'Stroke', 'Depression', 'INA_AD8', 'ADL', 'AMT', 'Intellectual_1',
       'Intellectual_2', 'Intellectual_3', 'Intellectual_4', 'Intellectual_5',
       'Intellectual_6', 'Intellectual_7', 'Intellectual_8', 'Social_1',
       'Social_2', 'Social_3', 'Social_4', 'Social_5', 'Social_6', 'Social_7',
       'Social_8', 'Social_9', 'Recreational_1', 'Recreational_2',
       'Recreational_3', 'Recreational_4', 'Recreational_5', 'Recreational_6',
       'Recreational_7', 'Recreational_8', 'Physical_1', 'Physical_2',
       'Physical_3', 'Physical_4', 'Physical_5', 'Intellectually_active',
       'Socially_active', 'Recreationally_act

In [3]:
# split into input (X) and output (y) variables
X = dta[['Education_category', 'Sex', 'Marital_status',
       'Income', 'Occupation_class', 'Living_area', 'Hypertension',
       'BMI_category', 'Diabetes', 'Smoking_status',  'Stroke', 'Depression',
       'Intellectual_1',
       'Intellectual_2', 'Intellectual_3', 'Intellectual_4', 'Intellectual_5',
       'Intellectual_6', 'Intellectual_7', 'Intellectual_8', 'Social_1',
       'Social_2', 'Social_3', 'Social_4', 'Social_5', 'Social_6', 'Social_7',
       'Social_8', 'Social_9', 'Recreational_1', 'Recreational_2',
       'Recreational_3', 'Recreational_4', 'Recreational_5', 'Recreational_6',
       'Recreational_7', 'Recreational_8', 'Physical_1', 'Physical_2',
       'Physical_3', 'Physical_4', 'Physical_5', 'Intellectually_active',
       'Socially_active', 'Recreationally_active', 'Physically_active',
       'Total_active', 'Carbohydrate', 'Carbo_category', 'Protein',
       'Protein_category', 'Vegetable', 'Vegetable_category', 'Fruit',
       'Fruit_category', 'Salted_fish', 'Salted_fish_category',
       'Instant_noodle', 'Instant_noodle_category', 'Tempe', 'Tempe_category']]
y = dta[['Category']]

Excluded age_category above

In [4]:
# format all fields as string
X = X.astype(str)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [6]:
# prepare input data
def prepare_inputs(X_train, X_test):
    oe = OrdinalEncoder()
    oe.fit(X_train)
    X_train_enc = oe.transform(X_train)
    X_test_enc = oe.transform(X_test)
    return X_train_enc, X_test_enc

In [7]:
# prepare target
def prepare_targets(y_train, y_test):
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_test_enc = le.transform(y_test)
    return y_train_enc, y_test_enc

In [8]:
# prepare input data
X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)
# prepare output data
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)

In [9]:
fs = SelectKBest(score_func=chi2, k='all')
fs.fit(X_train, y_train)
X_train_fs = fs.transform(X_train)
X_test_fs = fs.transform(X_test)

ValueError: could not convert string to float: '0-6 years'